<a href="https://www.kaggle.com/code/sikarosal/model-real-korean2?scriptVersionId=188451067" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import pandas as pd

df = pd.read_csv('/kaggle/input/data-final/data_final.csv')

In [2]:
# !pip install -r /kaggle/input/requirements/requirements.txt

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 143223 entries, 0 to 143222
Data columns (total 16 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   _id                     143223 non-null  object
 1   mediaType               143223 non-null  object
 2   gender                  143223 non-null  object
 3   address                 143223 non-null  object
 4   disasterLarge           143223 non-null  object
 5   disasterMedium          143223 non-null  object
 6   urgencyLevel            143223 non-null  object
 7   sentiment               143223 non-null  object
 8   symptom                 143223 non-null  object
 9   triage                  93451 non-null   object
 10  id                      143223 non-null  object
 11  json_file_path          143223 non-null  object
 12  wav_original_file_path  143223 non-null  object
 13  text                    143223 non-null  object
 14  endAt                   143223 non-n

In [4]:
import os
import sys

# 업로드한 파일들이 있는 디렉토리 경로 추가
sys.path.append('/kaggle/input/codes-for-model')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 143223 entries, 0 to 143222
Data columns (total 16 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   _id                     143223 non-null  object
 1   mediaType               143223 non-null  object
 2   gender                  143223 non-null  object
 3   address                 143223 non-null  object
 4   disasterLarge           143223 non-null  object
 5   disasterMedium          143223 non-null  object
 6   urgencyLevel            143223 non-null  object
 7   sentiment               143223 non-null  object
 8   symptom                 143223 non-null  object
 9   triage                  93451 non-null   object
 10  id                      143223 non-null  object
 11  json_file_path          143223 non-null  object
 12  wav_original_file_path  143223 non-null  object
 13  text                    143223 non-null  object
 14  endAt                   143223 non-n

In [6]:
df.drop('label',axis=1,inplace=True)

In [7]:
df['label']=df['urgencyLevel']

In [8]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df['label']=le.fit_transform(df['label'])
df['label']

0         1
1         1
2         1
3         2
4         2
         ..
143218    0
143219    0
143220    0
143221    0
143222    0
Name: label, Length: 143223, dtype: int64

In [9]:
df=df.drop(columns=['_id','triage','mediaType','gender','address',
                 'disasterLarge','disasterMedium','urgencyLevel','symptom','sentiment'],axis=1)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 143223 entries, 0 to 143222
Data columns (total 6 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   id                      143223 non-null  object
 1   json_file_path          143223 non-null  object
 2   wav_original_file_path  143223 non-null  object
 3   text                    143223 non-null  object
 4   endAt                   143223 non-null  int64 
 5   label                   143223 non-null  int64 
dtypes: int64(2), object(4)
memory usage: 6.6+ MB


In [11]:
from model import *

In [12]:
from collections import OrderedDict

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from transformers import AutoModel

class Baseline(nn.Module):
    def __init__(self,
                 model_link='beomi/KcELECTRA-base-v2022',
                 class_num=3):
        super(Baseline, self).__init__()
        self.electra = AutoModel.from_pretrained(model_link)
        
        self.classifier = nn.Sequential(OrderedDict([
            ('dense',nn.Linear(768, 768)),
            ('dropout', nn.Dropout(0.1)),
            ('out_proj', nn.Linear(768, class_num)),
        ]))
        
    def encode(self, input_ids, att_mask, token_type_ids):
        output = self.electra(input_ids, att_mask, token_type_ids)
        last_hidden_state = output.last_hidden_state
        
        # cls = torch.mean(last_hidden_state, dim=1)
        cls = last_hidden_state[:, 0, :]
        return cls
        
    def forward(self, input_ids, att_mask, token_type_ids):
        output = self.electra(input_ids, att_mask, token_type_ids)
        last_hidden_state = output.last_hidden_state
        
        # cls = torch.mean(last_hidden_state, dim=1)
        cls = last_hidden_state[:, 0, :]
        logit = self.classifier(cls)
        return logit

def get_Model(class_name):
    try:
        Myclass = eval(class_name)()
        return Myclass
    except NameError as e:
        print("Class [{}] is not defined".format(class_name))

def main():
    pass

if __name__ == "__main__":
    main()

In [ ]:
# https://github.com/HideOnHouse/TorchBase

import os
import glob
import wandb
import pickle
import argparse
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split, RandomSampler

from transformers import AutoTokenizer
from transformers import get_cosine_schedule_with_warmup
from transformers import get_linear_schedule_with_warmup

from dataset import *
from learning import *

from data_processing import *
# from inference import *
# from inference_switching_recall import *
from utils import DataParallelModel, DataParallelCriterion
from utils import set_device, set_save_path, set_label_frequency, str2bool, calc_metric

import warnings
warnings.filterwarnings(action='ignore')

SEED = 42 # 17
# random.seed(SEED) #  Python의 random 라이브러리가 제공하는 랜덤 연산이 항상 동일한 결과를 출력하게끔
os.environ['PYTHONHASHSEED'] = str(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

def main():
    is_scratch = True
    
    # Define project
    WANDB_AUTH_KEY = '8f36d8b64d0df471fb52315584870902fa63618f'
    project_name = 'new_project'
    model_name = 'Only_Text_{}'.format('scratch' if is_scratch else 'ckpt')
    model_link = 'beomi/KcELECTRA-base-v2022' #'beomi/kcbert-base'

    wandb.login(key=WANDB_AUTH_KEY)
    wandb.init(project=project_name)
    wandb.run.name = model_name

    # args
    epochs = 7
    batch_size = 48 # 128 + 64 + 32
    electra_lr = 2e-5
    cls_lr = 2e-3
    
    class_num = 3
    max_length = 384 # 384
    padding = 'max_length'
    upsampling_rate = 0.0
    by_file = True
    save_term = 100
    
    def set_device(main_device_num=0, using_device_num=1):
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
        device_ids = list(range(main_device_num, main_device_num + using_device_num))
        if device == 'cuda':
            device += ':{}'.format(main_device_num)
        print(device,device_ids)
        return device, device_ids
    
    main_device, device_ids = set_device(main_device_num=0, using_device_num=2)
    save_path = set_save_path(model_name, epochs, batch_size)

    # Datasets
    # train_path, valid_path, test_path = data_processing(root_path=os.path.join('파일 path 입력'), 
    #                                                     save_path=os.path.join('파일 저장 경로 입력'))
    # data_propressing 함수에 문제가 발생할 경우, 해당 line 주석처리 후 아래 주석의 코드를 실행
    
    y = df['label']  # 레이블 컬럼을 y로 지정

    x_train, x_valid = train_test_split(df, test_size=0.2, random_state=32, stratify=y)

    train_data = x_train
    valid_data = x_valid
    valid_file_ids = valid_data.id

    valid_file_ids = valid_data.id

    ## your Data Pre-Processing
    print('init Data >>>')
    print('\ttrain data :', train_data.shape)
    print('\tvalid data :', valid_data.shape)

    train_data = train_data.dropna(axis=0)
    train_data = train_data.reset_index(drop=True)
    valid_data = valid_data.dropna(axis=0)
    valid_data = valid_data.reset_index(drop=True)

    print('Drop nan >>>')
    print('\ttrain data :', train_data.shape)
    print('\tvalid data :', valid_data.shape)
    
    # print('Only One Sequence >>>')
    # train_data = set_last_sequence(train_data, end_time=120000, cut=False)
    # valid_data = set_last_sequence(valid_data, end_time=120000, cut=False)
    # test_data = set_last_sequence(test_data, end_time=120000, cut=True)
    # print('\ttrain data :', train_data.shape)
    # print('\tvalid data :', valid_data.shape)
    # print('\ttest data :', test_data.shape)

    print(f'Up-Sampling Label 0 >>> rate : {upsampling_rate}')
    train_data, train_label_frequency = set_label_frequency(train_data, rate=upsampling_rate, target_label=1, by_file=by_file)
    valid_data, valid_label_frequency = set_label_frequency(valid_data, rate=0.0, target_label=1, by_file=by_file)
  
    print('\ttrain data :', train_data.shape)
    print('\tvalid data :', valid_data.shape)
    
    ## Create Dataset and DataLoader
    tokenizer = AutoTokenizer.from_pretrained(model_link)
    train_dataset = MyDataset(train_data, 
                              tokenizer, 
                              max_length=max_length, 
                              padding=padding,
                              class_num=class_num)
    valid_dataset = MyDataset(valid_data,
                              tokenizer,
                              max_length=max_length,
                              padding=padding,
                              class_num=class_num)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, sampler=RandomSampler(train_dataset), num_workers=4)
    valid_loader = DataLoader(valid_dataset, batch_size=batch_size, num_workers=4)

    ## label_frequency
    train_label_frequency = (train_data.label == 1).sum() / len(train_data)
    valid_label_frequency = (valid_data.label == 1).sum() / len(valid_data)
    print("Label frequency of Train Data: {:6f}".format(train_label_frequency))
    print("Label frequency of Valid Data: {:6f}".format(valid_label_frequency))
    
    # modeling
    if is_scratch:
        model = Baseline(model_link=model_link, class_num=3)
        print(device_ids)
        model = DataParallelModel(model, device_ids=device_ids)#; model.to(device)
    else:
        check_path = os.path.join('models', 'One_Sequence_2m_O_72%_max_seq_256_e20_bs192', 'checkpoint_2_200.tar')
        model = Baseline(model_link=model_link, class_num=3)
        ckpt = torch.load(check_path, map_location=main_device)
        model.load_state_dict(ckpt['model_state_dict']); model.to(main_device)
        model = DataParallelModel(model, device_ids=device_ids)

    optimizer = optim.AdamW([{'params': model.module.electra.parameters(),'lr': electra_lr},
                             {'params': model.module.classifier.parameters(),'lr': cls_lr}],
                            eps=1e-8)
    criterion = torch.nn.CrossEntropyLoss()
    criterion = DataParallelCriterion(criterion, device_ids=device_ids)
    
    iter_len = len(train_loader)
    num_training_steps = iter_len * epochs
    num_warmup_steps = int(0.15 * num_training_steps)
    scheduler = get_linear_schedule_with_warmup(optimizer,
                                                num_warmup_steps=num_warmup_steps,
                                                num_training_steps=num_training_steps)

    config = {
        'electra_lr': electra_lr,
        'cls_lr': cls_lr,
        'batch_size': batch_size,
        'epochs': epochs,
        'max_length': max_length,
        'train_label_frequency': train_label_frequency,
        'valid_label_frequency': valid_label_frequency,
        'upsampling_rate': upsampling_rate,
        'by_file':by_file
    }
    wandb.config.update(config)

    # Train
    print("============================= Train =============================")
    model = train(train_label_frequency, scheduler, model, main_device, optimizer, criterion, epochs, save_path, train_loader, valid_loader, save_term)

if __name__ == '__main__':
    main()
    # CUDA_VISIBLE_DEVICES=0,1,2,3 python main.py
    # CUDA_VISIBLE_DEVICES=1,2,3,4 python main.py
    # CUDA_VISIBLE_DEVICES= python main.py

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: bellkjtt (sikaro). Use `wandb login --relogin` to force relogin


cuda:0 [0, 1]
init Data >>>
	train data : (114578, 6)
	valid data : (28645, 6)
Drop nan >>>
	train data : (114578, 6)
	valid data : (28645, 6)
Up-Sampling Label 0 >>> rate : 0.0
	train data : (114578, 6)
	valid data : (28645, 6)


tokenizer_config.json:   0%|          | 0.00/288 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/504 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/450k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Label frequency of Train Data: 0.335134
Label frequency of Valid Data: 0.339012


pytorch_model.bin:   0%|          | 0.00/511M [00:00<?, ?B/s]

[0, 1]
============================= Train =============================
100%|██████████| 597/597 [05:59<00:00,  1.66it/s, loss=0.9220, acc=0.5226]
val loss : 0.921991
val acc : 0.523
val acc(th) : 0.522604
val AUROC : 0.7700
val AUPRC : 0.6269
val Recall : 0.522271
val Precision : 0.5461
val F1_score : 0.4262
val Brier : 0.186159

100%|██████████| 597/597 [06:00<00:00,  1.66it/s, loss=0.8820, acc=0.5832]
val loss : 0.882026
val acc : 0.583
val acc(th) : 0.583173
val AUROC : 0.7834
val AUPRC : 0.6477
val Recall : 0.589033
val Precision : 0.5811
val F1_score : 0.5791
val Brier : 0.175349

 71%|███████   | 1701/2388 [47:05<19:03,  1.66s/it, epoch=3/7, loss=0.8227, acc=0.6061] 

In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
df['label'].unique()